# Smoke dtection Alarm Prediction(Classification)

# Main objective of the analysis

1)To predict accurately whether the fire alarm would go off given certain circumstances, which are features of the dataset, like Temperature,Humidity,eCO2,Pressure etc.

# Brief description of the data set 

In [ ]:
#pip install missingno

In [ ]:
import pandas as pd
import numpy as np
import missingno as msno
%matplotlib inline
import seaborn as sns

from sklearn.preprocessing import MinMaxScaler
sns.set_style('darkgrid')

In [ ]:
data = pd.read_csv('smoke_detection_iot.csv')
data.head()

In [ ]:
features = [p for p in data]
features

In [ ]:
data.describe()

In [ ]:
print(data.shape)

Examine the data types--there are many columns, so it might be wise to use value counts.
Determine if the floating point values need to be scaled.
Determine the breakdown of each activity.
Encode the activity label as an integer.

# Brief summary of data exploration

1.Checking for missing values:

In [ ]:
msno.bar(data)

There are no missing values.

In [ ]:
data.dtypes.value_counts()

We don't need to do any encoding as all data values are numeric.

We drop the columns ['Unnamed: 0', 'UTC','CNT'],that seem to have no bearing on the target values(Fire Alarm) and separate the features into X and labels(Fire Alarm) into y.

In [ ]:
X=data.drop(['Unnamed: 0', 'UTC','CNT'], axis=1)
  
y=data['Fire Alarm']
feature_cols = [x for x in X]

We scale the data for classifiers we are going to use, like KNN, that count Euclidean distance between data points. We use  MinMaxScaler from sklearn.preprocessing library to fit and transform X or features.

In [ ]:
scaler = MinMaxScaler()
X = scaler.fit_transform(X)
print(f"The range of feature inputs are within {X.min()} to {X.max()}")

In [ ]:
from sklearn.model_selection import StratifiedShuffleSplit

# Split the data into two parts with 1000 points in the test data
# This creates a generator
strat_shuff_split = StratifiedShuffleSplit(n_splits=4, test_size=0.2, random_state=42)

# Get the index values from the generator
train_idx, test_idx = next(strat_shuff_split.split(data[feature_cols], data['Fire Alarm']))

# Create the data sets
X_train = data.loc[train_idx, feature_cols]
y_train = data.loc[train_idx, 'Fire Alarm']

X_test = data.loc[test_idx, feature_cols]
y_test = data.loc[test_idx, 'Fire Alarm']


Compare the ratio of classes in both the train and test splits

In [ ]:
y_train.value_counts(normalize=True)

In [ ]:
y_test.value_counts(normalize=True)

# Summary of training at least three different classifier models¶

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
metrics = list()
def measure_error(y_test, y_pred, label):
    return pd.Series({'accuracy':accuracy_score(y_test, y_pred),
                      'precision': precision_score(y_test, y_pred),
                      'recall': recall_score(y_test, y_pred),
                      'f1': f1_score(y_test, y_pred)},
                      name=label)

1.LOGISTIC REGRESSION(L1 and L2)

In [ ]:
 from sklearn.linear_model import LogisticRegressionCV

# L1 regularized logistic regression
lr_l1 = LogisticRegressionCV(Cs=10, cv=4, penalty='l1', solver='liblinear',max_iter=10000).fit(X_train, y_train)
lr_l1.fit(X_train, y_train)
lr_l1_preds = lr_l1.predict(X_test)
#measure_error(y_test, lr_l1_preds, 'lr_l1')
metrics.append(measure_error(y_test, lr_l1_preds, 'lr_l1'))

In [ ]:
lr_l2 = LogisticRegressionCV(Cs=10, cv=4, penalty='l2', solver='liblinear',max_iter=10000).fit(X_train, y_train)
lr_l2.fit(X_train, y_train)
lr_l2_preds = lr_l2.predict(X_test)
metrics.append(measure_error(y_test, lr_l2_preds, 'lr_l2'))

KNN

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors=5)
knn = knn.fit(X_train, y_train)
knn_pred = knn.predict(X_test)
metrics.append(measure_error(y_test, knn_pred, 'knn'))

RANDOM FOREST CLASSIFIER

In [ ]:
from sklearn.ensemble import RandomForestClassifier
n_estimators=20
M_features=X.shape[1]
max_features=round(np.sqrt(M_features))-1
rf = RandomForestClassifier( max_features=max_features,n_estimators=20, random_state=0)
rf.fit(X_train,y_train)
rf_pred = rf.predict(X_test)
metrics.append(measure_error(y_test, rf_pred, 'rf'))

A COMPARISON OF ALL CLASSIFIERS 

In [ ]:
metrics = pd.concat(metrics, axis=1)
metrics.to_clipboard()
metrics

In [ ]:
def visualize_eval_metrics(metrics):
    df = pd.DataFrame(data=metrics)
    #table = pd.pivot_table(df, values='type', index=['accuracy', 'precision', 'recall', 'f1', 'auc'],
    #                columns=['type'])
    #df = df.set_index('type').transpose()
    print(df)
    x = np.arange(5)
    original = df.iloc[0, 1:].values
    class_weight = df.iloc[1, 1:].values
    smote = df.iloc[2, 1:].values
    under = df.iloc[3, 1:].values
    width = 0.2
    figure(figsize=(12, 10), dpi=80)
    plt.bar(x-0.2, original, width, color='#95a5a6')
    plt.bar(x, class_weight, width, color='#d35400')
    plt.bar(x+0.2, smote, width, color='#2980b9')
    plt.bar(x+0.4, under, width, color='#3498db')
    plt.xticks(x, ['Accuracy', 'Recall', 'AUC', 'Precision', 'Fscore'])
    plt.xlabel("Evaluation Metrics")
    plt.ylabel("Score")
    plt.legend(["Original", "Class Weight", "SMOTE", "Undersampling"])
    plt.show()

# A paragraph explaining which of your classifier models you recommend

In [ ]:
On the accompanying pdf file.

# Summary Key Findings and Insights

In [ ]:
On the accompanying pdf file.

# Suggestions for next steps in analyzing this data

In [ ]:
On the accompanying pdf file.